| [$\leftarrow$ Polynomial Regression ](n3_polynomial_regression.ipynb) | [Comparison and Conclusion $\rightarrow$](n5_comparison_conclusion.ipynb) |
| :-----------------------------------------------------------------: | :---------------------------------------------------------------: |

<hr>

### <hr>

| [$\leftarrow$ Polynomial Regression ](n3_polynomial_regression.ipynb) | [Comparison and Conclusion $\rightarrow$](n5_comparison_conclusion.ipynb) |
| :-----------------------------------------------------------------: | :---------------------------------------------------------------: |
